<a href="https://colab.research.google.com/github/hpadhuka/customizing_models_tf2/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input, Dense, LSTM, Masking
from tensorflow.keras.models import Model
import random
import matplotlib.pyplot as plt

![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [2]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
#NUM_EXAMPLES = 500

data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            print(line)
            print(line.split('\t'))
            break

FileNotFoundError: ignored

In [ ]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [ ]:
english_sentences =[]
german_sentences = []
for sentence in data_examples:
    eng, deu, rest = sentence.split('\t')
    eng = preprocess_sentence(eng)
    deu = preprocess_sentence(deu)
    english_sentences.append(eng)
    german_sentences.append("<start> {0} <end>".format(deu))

In [ ]:
def create_tokenizer(list_of_strings):
    """
    Word tokenizer 
    """
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', char_level=False, lower=False)
    tokenizer.fit_on_texts(list_of_strings)
    return tokenizer



def strings_to_sequences(tokenizer, list_of_strings):
    """
    This function takes a tokenizer object and a list of strings as its arguments.
    It should use the tokenizer to map the text chunks to sequences of tokens and
    then return this list of encoded sequences.
    """
    
    return tokenizer.texts_to_sequences(list_of_strings)

tokenizer = create_tokenizer(german_sentences)
deu_input_dim = len(tokenizer.word_index) + 1
print(deu_input_dim)
german_seq_chunks = strings_to_sequences(tokenizer, german_sentences)

In [ ]:
num_samples = 5
inx = np.random.choice(len(english_sentences), num_samples, replace=False)
for idx in inx:
    print(english_sentences[idx], '\t', german_sentences[idx], '\t', german_seq_chunks[idx])

In [ ]:
def make_padded_dataset(sequence_chunks):
    """
    This function takes a list of lists of tokenized sequences, and transforms
    them into a 2D numpy array, padding the sequences as necessary according to
    the above specification. The function should then return the numpy array.
    """
    
    return tf.keras.preprocessing.sequence.pad_sequences(sequence_chunks, maxlen=20, padding='post', value=0.0)

In [ ]:
# Pad the token sequence chunks and get the numpy array

german_padded_sequences = make_padded_dataset(german_seq_chunks)
print(german_padded_sequences.shape)

## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [ ]:
# Load embedding module from Tensorflow Hub

embedding_layer = hub.KerasLayer("./models/tf2-preview_nnlm-en-dim128_1", 
                                 output_shape=[128], input_shape=[], dtype=tf.string)

In [ ]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape


You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [ ]:
training_data, test_data, training_label, test_label = train_test_split(english_sentences, german_padded_sequences, test_size=0.2)
print(training_data[0])
print(training_label[0])
#print(training_data[0], len(training_data))
#print(training_val[0], len(training_val))
#print(test_data[0], len(test_data))
#print(test_val[0], len(test_val))

In [ ]:
def make_dataset(input_array, target_array):
    """
    This function takes two 2D numpy arrays in the first two arguments, and an integer
    batch_size in the third argument. It should create and return a Dataset object 
    using the two numpy arrays and batch size according to the above specification.
    """
    
    dataset = tf.data.Dataset.from_tensor_slices((input_array, target_array))
    #dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

In [ ]:

train_dataset = make_dataset(training_data, training_label)
val_dataset = make_dataset(test_data, test_label)



def split(x, y):
    split_x = tf.strings.split(x)
    return (split_x, y)

def embed(x, y):
    return (embedding_layer(x), y)

def filter_long(x, y):
    return len(x) < 14



def pad(x, y):
    paddings = tf.constant([[13, 0], [0, 0]])
    new_x = tf.pad(x, paddings, "CONSTANT")
    return (new_x[-13:,:], y)
    
split_train_dataset = train_dataset.map(split)
split_val_dataset = val_dataset.map(split)

embed_train_dataset = split_train_dataset.map(embed)
embed_val_dataset = split_val_dataset.map(embed)

filter_train_dataset = embed_train_dataset.filter(filter_long)
filter_val_dataset = embed_val_dataset.filter(filter_long)


pad_train_dataset = filter_train_dataset.map(pad)
pad_val_dataset = filter_val_dataset.map(pad)

batch_train = pad_train_dataset.batch(16, drop_remainder=True)
batch_val = pad_val_dataset.batch(16, drop_remainder=True)

print(batch_train.element_spec)
print(batch_val.element_spec)

for ds in batch_train.take(1):
    print("English training example shape:", ds[0].shape)

for ds in batch_val.take(1):
    print("German validation example shape:", ds[1].shape)
  

## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [ ]:
class EndToken(Layer):
    def __init__(self):
        super(EndToken, self).__init__()
        init = tf.random_normal_initializer()
        self.end_token = tf.Variable(initial_value=init(shape=(1, 1, 128)))
    def call(self, inputs):
        #print("Inputs shape:", inputs.shape)
        batch_size = 16
        if inputs.shape[0]:
            batch_size = inputs.shape[0]
        #print(self.end_token.shape)
        tile_end_token = tf.tile(self.end_token, [batch_size, 1, 1])
        concat_result = tf.concat([inputs, tile_end_token], axis=1)
        return concat_result
                                     

In [ ]:
et = EndToken()
for ds_eng, ds_deu in batch_train.take(1):
    print("English training example shape:", ds_eng.shape)
    output = et(ds_eng)
    print("Shape after end token concatenation", output.shape)
    

## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

In [ ]:
inputs = Input(shape=(13, 128))
output = EndToken()(inputs)
output = Masking()(output)
output = LSTM(512, return_state=True)(output)

encoder = Model(inputs=inputs, outputs=output, name="encoder")
encoder.summary()

for ds_eng, ds_deu in batch_train.take(1):
    final_output, final_eng_hidden_state, final_eng_cell_state = encoder(ds_eng)
    print(f'Unused final output step shape: {final_output.shape}')
    print(f'Hidden state: {final_eng_hidden_state.shape}')
    print(f'Cell state: {final_eng_cell_state.shape}')


## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [ ]:

class Decoder(Model):
    def __init__(self, **kwargs):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(input_dim=deu_input_dim , output_dim=128, mask_zero=True)
        self.lstm = LSTM(512, return_sequences=True, return_state=True)
        self.dense = Dense(deu_input_dim)
        
    def call(self, inputs, hidden_state=None, cell_state=None):
        embed_inputs = self.embedding(inputs)
        #if len(embed_inputs.shape.as_list()) == 1:
        #    embed_inputs = tf.expand_dims(embed_inputs, axis=0)
        #if len(hidden_state.shape.as_list()) == 2:
        #    hidden_state = tf.expand_dims(hidden_state, axis=0)
        #if len(cell_state.shape.as_list()) == 2:
        #    cell_state = tf.expand_dims(cell_state, axis=0)
            
        lstm_output, current_hidden_state, current_cell_state = self.lstm(embed_inputs, initial_state=[hidden_state, cell_state])
        outputs = self.dense(lstm_output)
        return outputs, current_hidden_state, current_cell_state
    
    

In [ ]:
for ds_eng, ds_deu in batch_train.take(1):
    output, final_eng_hidden_state, final_eng_cell_state = encoder(ds_eng)
    d = Decoder(name="decoder")
    seq_output, hidden_state, cell_state = d(ds_deu, final_eng_hidden_state, final_eng_cell_state)
    print(seq_output.shape)
    d.summary()

## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [ ]:
def prepare_train_data(deu_batch):
    batch_inputs = []
    batch_outputs = []
    for row in deu_batch:
        y = tf.where(tf.equal(row, 2)).numpy()[0][0]
        row1 = row[:y]
        row2 = row[1:y+1]
        size = 20 - tf.size(row1).numpy()
        paddings = tf.constant([[0, size]])
        row1 = tf.pad(row1, paddings)
        row2 = tf.pad(row2, paddings)
        batch_inputs.append(row1)
        batch_outputs.append(row2)
    
    batch_inputs = tf.stack(batch_inputs, 0)
    batch_outputs = tf.stack(batch_outputs, 0)
    return(batch_inputs, batch_outputs)

for eng_data, deu_data in batch_train.take(1):
    eng_input = eng_data
    deu_input, deu_output = prepare_train_data(deu_data)
    print(deu_input.shape)
    print(deu_input)

In [ ]:
def loss_function(real, pred):
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss = cross_entropy(y_true=real, y_pred=pred)
  mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
  mask = tf.cast(mask, dtype=loss.dtype)  
  loss = mask * loss
  loss = tf.reduce_mean(loss)
  return loss

In [ ]:
#tf.config.experimental_run_functions_eagerly(True)
decoder = Decoder(name="decoder")
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def compute_pass(eng_input, deu_input, deu_output, training=True):    
    with tf.GradientTape() as tape:
        discard_output_seq, eng_final_hidden_state, eng_final_cell_state = encoder(eng_input)
        output_seq, hidden_state, cell_state = decoder(deu_input, eng_final_hidden_state, eng_final_cell_state)
        loss = loss_function(deu_output, output_seq)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    if training:
        optimizer.apply_gradients(zip(gradients, variables))

    return loss, gradients

#compute_pass(eng_input, deu_input, deu_output)


In [ ]:

def train_translator(num_epochs, train_dataset, val_dataset):
    """
    Returns training and validation losses per epoch
    """
    
    train_epoch_loss = []
    val_epoch_loss = []
    #val_shuffle_dataset = val_dataset.shuffle(320, reshuffle_each_iteration=True)
    #loss = tf.keras.losses.sparse_categorical_crossentropy(deu_output, output_seq, from_logits=True)
    
    for _ in range(num_epochs):
        train_batch_loss = []
        val_batch_loss = []
        # Iterate over the batches of the train dataset.
        for step, (eng_data, deu_data) in enumerate(train_dataset):
            eng_input = eng_data
            deu_input, deu_output = prepare_train_data(deu_data)
            current_loss, gradients = compute_pass(eng_input, deu_input, deu_output)
            print("epoch:", _, "step:", step, "train batch loss:", current_loss)
            train_batch_loss.append(current_loss)

        
        # Iterate over the batches of the val dataset.
        for step, (eng_data, deu_data) in enumerate(val_dataset):
            eng_input = eng_data
            deu_input, deu_output = prepare_train_data(deu_data)
            current_loss, gradients = compute_pass(eng_input, deu_input, deu_output, training=False)
            print("epoch:", _, "step:", step, "val batch loss:", current_loss)
            val_batch_loss.append(current_loss)

            
        train_mean_loss = np.mean(train_batch_loss)
        val_mean_loss = np.mean(val_batch_loss)
        train_epoch_loss.append(train_mean_loss)
        val_epoch_loss.append(val_mean_loss)
        print('Epoch {0} training loss: {1} val loss: {2}'.format(_, train_mean_loss, val_mean_loss))    
        
    return train_epoch_loss, val_epoch_loss

        

In [ ]:
# Train the model for 3 epochs

train_loss_results, val_loss_results = train_translator(3, batch_train, batch_val)

In [ ]:
print("Training loss per epoch:", train_loss_results)
print("Validation loss per epoch:", val_loss_results)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# set x axis intervals to 1
axes[0].xaxis.set_ticks(np.arange(0, len(train_loss_results), 1))
axes[1].xaxis.set_ticks(np.arange(0, len(train_loss_results), 1))

axes[0].set_xlabel("Epochs", fontsize=14)
axes[0].set_ylabel("Loss", fontsize=14)
axes[0].set_title('Training Loss vs epochs')
axes[0].plot(train_loss_results)

axes[1].set_xlabel("Epochs", fontsize=14)
axes[1].set_ylabel("Loss", fontsize=14)
axes[1].set_title('Val Loss vs epochs')
axes[1].plot(val_loss_results)
plt.show()

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.

In [ ]:
total_sent = len(data_examples)
random_samples = random.sample(range(0, total_sent), 100)

english_sentences =[]
embedded_eng_sentences = []

for sentence_index in random_samples:
    eng, deu, rest = data_examples[sentence_index].split('\t')
    eng = preprocess_sentence(eng)
    english_sentences.append(eng)

for sent in english_sentences:
    print(sent)
    eng_embedded_sent = embedding_layer(tf.strings.split(sent))
    paddings = tf.constant([[13, 0], [0, 0]])
    x = tf.pad(eng_embedded_sent, paddings, "CONSTANT")
    padded_eng_sent = x[-13:,:]
    #print(padded_eng_sent.shape)
    padded_eng_sent = tf.expand_dims(padded_eng_sent, axis=0)
    #print(padded_eng_sent.shape)
    #print(padded_eng_sent)
    discard_output_seq, eng_final_hidden_state, eng_final_cell_state = encoder(padded_eng_sent)
    #print(eng_final_hidden_state.shape)
    #print(eng_final_cell_state.shape)
    #print(tf.expand_dims(eng_final_hidden_state, axis=0).shape)
    deu_sentence = ""
    #output_seq, hidden_state, cell_state = decoder(tf.constant(deu_index), eng_final_hidden_state, eng_final_cell_state)
    #output_seq = tf.squeeze(output_seq)
    #print(np.argmax(output_seq))
    #seq = []
    #seq.append([np.argmax(output_seq)])
    #token = tokenizer.sequences_to_texts(seq)[0]
    #print(token)
    #break
    decoder_hidden_state, decoder_cell_state = eng_final_hidden_state, eng_final_cell_state
    token = start_token = input_sentence = '<start> something'
    #deu_index = tokenizer.texts_to_sequences([input_sentence])
    counter = 0
    while token != '<end>':
        #print(input_sentence)
        deu_index = tokenizer.texts_to_sequences([input_sentence])
        #print(deu_index)
        #print(deu_index)
        #print("deu_sent:", deu_sent)
        #print(deu_sent[0])
        #print(deu_sent)
        output_seq, decoder_hidden_state, decoder_cell_state = decoder(tf.constant(deu_index), decoder_hidden_state, decoder_cell_state)
        #print(output_seq.shape)
        #output_seq = tf.squeeze(output_seq)
        #print(output_seq.shape)
        #print(output_seq)
        #print(output_seq[:,-1])
        #print(np.argmax(output_seq))
        seq = []
        #deu_index[0][0] = "{0} {1}".format(deu_index[0][0], np.argmax(output_seq[:,-1]))
        seq.append([np.argmax(output_seq[:,-1])])
        token = tokenizer.sequences_to_texts(seq)[0]
        #print("TOKEN:", token)
        #print([np.argmax(output_seq[:,-1])])
        #print(tokenizer.sequences_to_texts(deu_index))
        #print(seq)
        #print(tokenizer.sequences_to_texts(seq))
        #token = tokenizer.sequences_to_texts(seq)[0]
        #if token == '':
        #    continue
        #print(token)
        if token != '<end>':
            deu_sentence += " {0}".format(token)
            input_sentence = "{0}{1}".format(start_token, deu_sentence)
        counter += 1
        if counter > 12:
            break
    print(deu_sentence)
        #discard_output_seq, eng_final_hidden_state, eng_final_cell_state = encoder(padded_eng_sent)
        #print(eng_final_hidden_state.shape, eng_final_cell_state.shape)
        #output_seq, hidden_state, cell_state = decoder(embedding_layer(tf.constant(["start"])), eng_final_hidden_state, eng_final_cell_state)
        #break


In [ ]:
x = tf.keras.layers.Embedding(input_dim=deu_input_dim , output_dim=128, mask_zero=True)
y = x(tf.constant([[1, 4]]))
print(y.shape)
print(y[:,-1,:])

In [ ]:
tf.constant([1, 4])

In [ ]:
tokenizer.texts_to_sequences(["<start> ich dich"])